# Data Exploration

Loading in the data and learning more about what we have access to

In [22]:
import pandas as pd
from collections import defaultdict

In [3]:
leagues_df = pd.read_json("esports-data/leagues.json")
players_df = pd.read_json("esports-data/players.json")
teams_df = pd.read_json("esports-data/teams.json")
tournaments_df = pd.read_json("esports-data/tournaments.json")

In [10]:
single_game_df = pd.read_json("games/ESPORTSTMNT03_3195064.json")

game_info = single_game_df[single_game_df["eventType"] == "game_info"]

In [23]:
def get_team_players(game_info):
    participants = game_info["participants"]
    team = defaultdict(list)
    for participant in participants:
        for info in participant:
            team[info["teamID"]].append(info)

    return team

In [41]:
set(single_game_df["eventType"])

{'building_destroyed',
 'champion_kill',
 'champion_kill_special',
 'champion_level_up',
 'epic_monster_kill',
 'epic_monster_spawn',
 'game_end',
 'game_info',
 'item_destroyed',
 'item_purchased',
 'item_sold',
 'item_undo',
 'objective_bounty_prestart',
 'queued_dragon_info',
 'queued_epic_monster_info',
 'skill_level_up',
 'stats_update',
 'summoner_spell_used',
 'turret_plate_destroyed',
 'turret_plate_gold_earned',
 'ward_killed',
 'ward_placed'}

In [89]:
teams_to_players = get_team_players(game_info)
players_to_team = {}
for team_id,info in teams_to_players.items():
    for player in info:
        players_to_team[player["participantID"]] = player["teamID"]

In [90]:
# I want to have a dataframe that holds the following:
"""
        numKills | numWards | firstBlood | turretsDestroyed | turretPlateGold | numItems | numDragsKilled | numBarons
team1: 
team2:
"""

team_ids = list(set(teams_to_players.keys()))
model_dict = {
    'teamID': team_ids,
    'numKills': ["",""],
    'numWards': ["",""],
    'firstBlood': ["",""],
    'turretsDestroyed': ["",""],
    'turretPlateGold': ["",""],
    'numItems': ["",""],
    "numDragsKilled": ["",""],
    'numBarons': ["",""],
}

In [91]:
# Get numKills
numKills = defaultdict(int)
kill_info = single_game_df[single_game_df["eventType"] == "champion_kill"]

for index, row in kill_info.iterrows():
    numKills[row["killerTeamID"]] += 1

model_dict["numKills"] = list(numKills.values())

model_dict

{'teamID': [200, 100],
 'numKills': [15, 27],
 'numWards': ['', ''],
 'firstBlood': ['', ''],
 'turretsDestroyed': ['', ''],
 'turretPlateGold': ['', ''],
 'numItems': ['', ''],
 'numDragsKilled': ['', ''],
 'numBarons': ['', '']}

In [92]:
#Get numWards
numWards = defaultdict(int)
ward_info = single_game_df[single_game_df["eventType"] == "ward_placed"]

for index, row in ward_info.iterrows():
    numWards[players_to_team[row["placer"]]] += 1

model_dict["numWards"] = list(numWards.values())

model_dict

{'teamID': [200, 100],
 'numKills': [15, 27],
 'numWards': [51, 41],
 'firstBlood': ['', ''],
 'turretsDestroyed': ['', ''],
 'turretPlateGold': ['', ''],
 'numItems': ['', ''],
 'numDragsKilled': ['', ''],
 'numBarons': ['', '']}

In [104]:
#Get firstBlood
firstBlood = defaultdict(int)

firstBlood_df = single_game_df[single_game_df["killType"] == "firstBlood"]
firstBlood_df = firstBlood_df[firstBlood_df.columns[~firstBlood_df.isnull().all()]]

for index, row in firstBlood_df.iterrows():
    firstBlood[players_to_team[row["killer"]]] += 1

model_dict["firstBlood"] = list(firstBlood.values())

model_dict

{'teamID': [200, 100],
 'numKills': [15, 27],
 'numWards': [51, 41],
 'firstBlood': [1],
 'turretsDestroyed': ['', ''],
 'turretPlateGold': ['', ''],
 'numItems': ['', ''],
 'numDragsKilled': ['', ''],
 'numBarons': ['', '']}